# Bronze.csv

Energia grupowana po regionie i czasie + informacje o regionie + informacje o sasiadach. LSTM, Dense, XGB

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import xgboost as xgb
warnings.filterwarnings('ignore')

2023-09-22 16:40:15.098606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 16:40:16.903534: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../../data/data_world.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1930-12-08T08:01:02.000Z,23.261,120.277,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930-12-03T18:51:47.000Z,18.233,96.298,10.0,7.4,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1930-12-02T07:01:30.000Z,25.854,98.356,35.0,6.2,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1930-11-28T07:32:56.000Z,18.779,-106.767,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1930-11-25T19:02:53.000Z,35.050,139.129,15.0,6.9,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[["time", "latitude", "longitude", "mag"]]

In [4]:
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%dT%H:%M:%S.%fZ")
df

,time,latitude,longitude,mag
0,1930-12-08 08:01:02.000,23.2610,120.2770,6.3
1,1930-12-03 18:51:47.000,18.2330,96.2980,7.4
2,1930-12-02 07:01:30.000,25.8540,98.3560,6.2
3,1930-11-28 07:32:56.000,18.7790,-106.7670,6.3
4,1930-11-25 19:02:53.000,35.0500,139.1290,6.9
...,...,...,...,...
797041,2018-09-01 01:14:38.230,-30.4830,-177.9279,4.3
797042,2018-09-01 01:07:59.120,-10.7558,124.3621,4.0
797043,2018-09-01 01:00:13.810,-5.5167,147.1735,4.6
797044,2018-09-01 00:27:11.440,46.8819,155.6566,4.3


In [5]:
time_cut = dt.datetime(1973, 1, 1)
df = df[df["time"] > time_cut]
df["energy"] = 10**(1.44*df["mag"]+5.24)
df["label"] = np.where(df["mag"] > 5, 1, 0)

In [6]:
time_split = 30

In [7]:
# group the data into 30 days intervals
df["time"] = df["time"].dt.floor('d')
df["time"] = df["time"] - np.min(df["time"])
df["time"] = df["time"].dt.days // time_split
df["time"] = df["time"].astype(int)

In [8]:
geo_split = 10

In [9]:
df["latitude"] = df["latitude"] // geo_split
df["latitude"] = df["latitude"] + np.abs(np.min(df["latitude"]))

df["longitude"] = df["longitude"] // geo_split
df["longitude"] = df["longitude"] + np.abs(np.min(df["longitude"]))

In [10]:
df

,time,latitude,longitude,mag,energy,label
21801,12,7.0,34.0,4.5,5.248075e+11,0
21802,12,6.0,11.0,4.4,3.767038e+11,0
21803,12,7.0,0.0,5.1,3.837072e+12,1
21804,12,7.0,34.0,4.4,3.767038e+11,0
21805,12,6.0,34.0,4.6,7.311391e+11,0
...,...,...,...,...,...,...
797041,555,5.0,0.0,4.3,2.703958e+11,0
797042,555,7.0,30.0,4.0,1.000000e+11,0
797043,555,8.0,32.0,4.6,7.311391e+11,0
797044,555,13.0,33.0,4.3,2.703958e+11,0


In [11]:
df_agg = df.groupby(["latitude", "longitude", "time"]).agg({"energy": "sum", "label": "sum"}).reset_index()
df_agg

,latitude,longitude,time,energy,label
0,0.0,13.0,268,1.018591e+12,0
1,0.0,22.0,482,3.767038e+11,0
2,0.0,31.0,375,5.248075e+11,0
3,0.0,31.0,473,7.311391e+11,0
4,0.0,33.0,280,2.703958e+11,0
...,...,...,...,...,...
71216,17.0,30.0,487,1.018591e+12,0
71217,17.0,30.0,489,3.767038e+11,0
71218,17.0,30.0,509,7.311391e+11,0
71219,17.0,30.0,538,3.767038e+11,0


In [12]:
df_agg["label"] = np.where(df_agg["label"] > 0, 1, 0)

In [67]:
def make_ds(df_agg, block_size):
    dfs_train, dfs_val = [], []
    for i in df_agg["latitude"].unique():
        for j in df_agg["longitude"].unique():
            tmp = df_agg[(df_agg["latitude"] == i) & (df_agg["longitude"] == j)]
            if not tmp.empty:
                start = max(tmp["time"].min() - block_size, 0)
                end = min(tmp["time"].max() + block_size, df_agg["time"].max())
                tmp = tmp.set_index("time").reindex(range(start, end)).fillna(0).rename_axis('time').reset_index()
                tmp["label"] = tmp["label"].shift(-1)
                tmp["latitude"] = i
                tmp["longitude"] = j
                n = int(0.8 * len(tmp))
                df_train = tmp[:n]
                df_val = tmp[n:]
                scaler = MinMaxScaler()
                df_train["energy"] = scaler.fit_transform(df_train[["energy"]])
                df_val["energy"] = scaler.transform(df_val[["energy"]])
                for idx in range(1, block_size):
                    df_train["energy" + str(idx)] = df_train["energy"].shift(idx)
                    df_val["energy" + str(idx)] = df_val["energy"].shift(idx)      
                dfs_train.append(df_train)
                dfs_val.append(df_val)
    df_final_train = pd.concat(dfs_train)
    df_final_val = pd.concat(dfs_val)
    return df_final_train, df_final_val

In [68]:
block_size = 16
df_train, df_val = make_ds(df_agg, block_size)

In [69]:
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)

In [70]:
df_train

,time,latitude,longitude,energy,label,energy_ma_4,energy_ma_8,energy_ma_12,energy_ma_24,energy_ma_36,...,energy_ma_12_14,energy_ma_24_14,energy_ma_36_14,energy14,energy_ma_4_15,energy_ma_8_15,energy_ma_12_15,energy_ma_24_15,energy_ma_36_15,energy15
50,409,0.0,31.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.027778,...,0.000000,0.041667,0.027778,0.0,0.000000,0.000000,0.000000,0.041667,0.027778,0.0
51,410,0.0,31.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.027778,...,0.000000,0.041667,0.027778,0.0,0.000000,0.000000,0.000000,0.041667,0.027778,0.0
52,411,0.0,31.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.041667,0.027778,0.0,0.000000,0.000000,0.000000,0.041667,0.027778,0.0
53,412,0.0,31.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.041667,0.027778,0.0,0.000000,0.000000,0.000000,0.041667,0.027778,0.0
54,413,0.0,31.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.027778,0.0,0.000000,0.000000,0.000000,0.041667,0.027778,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,488,17.0,23.0,0.000000,0.0,0.000000,0.011464,0.007643,0.009145,0.006097,...,0.010647,0.005324,0.008494,0.0,0.031942,0.015971,0.010647,0.005324,0.008494,0.0
260,489,17.0,23.0,0.000000,0.0,0.000000,0.011464,0.007643,0.009145,0.006097,...,0.010647,0.005324,0.008494,0.0,0.031942,0.015971,0.010647,0.005324,0.008494,0.0
261,490,17.0,23.0,0.178002,0.0,0.044501,0.033714,0.022476,0.016562,0.011041,...,0.010647,0.005324,0.008494,0.0,0.031942,0.015971,0.010647,0.005324,0.008494,0.0
262,491,17.0,23.0,0.000000,0.0,0.044501,0.033714,0.022476,0.016562,0.011041,...,0.010647,0.005324,0.008494,0.0,0.000000,0.015971,0.010647,0.005324,0.008494,0.0


In [72]:
x_train = df_train.drop(["label", "time", "longitude", "latitude", "energy_ma_4", "energy_ma_8", "energy_ma_12", "energy_ma_24", "energy_ma_36"], axis=1).to_numpy()
y_train = df_train["label"].to_numpy()
x_val = df_val.drop(["label", "time", "longitude", "latitude", "energy_ma_4", "energy_ma_8", "energy_ma_12", "energy_ma_24", "energy_ma_36"], axis=1).to_numpy()
y_val = df_val["label"].to_numpy()

In [73]:
x_train = x_train.reshape(-1, block_size, 6)
x_val = x_val.reshape(-1, block_size, 6)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

In [74]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [75]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, input_shape=(block_size, 6), return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2),
])

In [76]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [77]:
model.fit(x_train, y_train, epochs=10, batch_size=512, validation_data=(x_val, y_val))

Epoch 1/10
297/297 [==============================] - 37s 94ms/step - loss: 0.3620 - accuracy: 0.8805 - f1_score: 0.4711 - val_loss: 0.4072 - val_accuracy: 0.8663 - val_f1_score: 0.4642
Epoch 2/10
297/297 [==============================] - 27s 92ms/step - loss: 0.3457 - accuracy: 0.8832 - f1_score: 0.4690 - val_loss: 0.3814 - val_accuracy: 0.8663 - val_f1_score: 0.4642
Epoch 3/10
297/297 [==============================] - 28s 94ms/step - loss: 0.3390 - accuracy: 0.8832 - f1_score: 0.4690 - val_loss: 0.4011 - val_accuracy: 0.8663 - val_f1_score: 0.4642
Epoch 4/10
297/297 [==============================] - 30s 103ms/step - loss: 0.3351 - accuracy: 0.8832 - f1_score: 0.4690 - val_loss: 0.3810 - val_accuracy: 0.8663 - val_f1_score: 0.4642
Epoch 5/10
297/297 [==============================] - 32s 108ms/step - loss: 0.3313 - accuracy: 0.8832 - f1_score: 0.4690 - val_loss: 0.3806 - val_accuracy: 0.8659 - val_f1_score: 0.4668
Epoch 6/10
297/297 [==============================] - 33s 111ms/step

In [78]:
x_train = df_train.drop(["label", "time", "longitude", "latitude", "energy_ma_4", "energy_ma_8", "energy_ma_12", "energy_ma_24", "energy_ma_36"], axis=1).to_numpy()
y_train = df_train["label"].to_numpy()
x_val = df_val.drop(["label", "time", "longitude", "latitude", "energy_ma_4", "energy_ma_8", "energy_ma_12", "energy_ma_24", "energy_ma_36"], axis=1).to_numpy()
y_val = df_val["label"].to_numpy()

In [79]:
x_train = x_train.reshape(-1, block_size, 6)
x_val = x_val.reshape(-1, block_size, 6)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

In [80]:
x_pos_train = df_train[["latitude", "longitude"]].to_numpy()
x_pos_val = df_val[["latitude", "longitude"]].to_numpy()

In [81]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [90]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed, vocab_size):
        super(MyModel, self).__init__()
        self.n_embed = n_embed
        self.loc_emb_h = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.loc_emb_c = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.lstm1 = tf.keras.layers.LSTM(n_embed * 2, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed * 2)
        self.dense = tf.keras.layers.Dense(n_embed, activation='relu')
        self.out = tf.keras.layers.Dense(2)
        

    def call(self, inputs):
        loc, timestep = inputs
        h = self.loc_emb_h(loc)
        c = self.loc_emb_c(loc)
        # reshape h and c to (batch_size, n_embed)
        h = tf.reshape(h, [-1, 2 * self.n_embed])
        c = tf.reshape(c, [-1, 2 * self.n_embed])
        x = self.lstm1(timestep, initial_state=[c, h])
        x = self.lstm2(x, initial_state=[c, h])
        x = self.dense(x)
        return self.out(x)

In [91]:
n_embed = 256
vocab_size = 2000
model = MyModel(n_embed, vocab_size)

In [92]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [93]:
model.fit([x_pos_train, x_train], y_train, epochs=40, batch_size=512, validation_data=([x_pos_val, x_val], y_val))

Epoch 1/40
297/297 [==============================] - 43s 125ms/step - loss: 0.2918 - accuracy: 0.8966 - f1_score: 0.6436 - val_loss: 0.2847 - val_accuracy: 0.8899 - val_f1_score: 0.7204
Epoch 2/40
297/297 [==============================] - 29s 98ms/step - loss: 0.2400 - accuracy: 0.9129 - f1_score: 0.7409 - val_loss: 0.2705 - val_accuracy: 0.8982 - val_f1_score: 0.7389
Epoch 3/40
297/297 [==============================] - 41s 138ms/step - loss: 0.2246 - accuracy: 0.9171 - f1_score: 0.7570 - val_loss: 0.2574 - val_accuracy: 0.9010 - val_f1_score: 0.7403
Epoch 4/40
297/297 [==============================] - 29s 98ms/step - loss: 0.2182 - accuracy: 0.9181 - f1_score: 0.7612 - val_loss: 0.2479 - val_accuracy: 0.9025 - val_f1_score: 0.7387
Epoch 5/40
297/297 [==============================] - 30s 100ms/step - loss: 0.2131 - accuracy: 0.9185 - f1_score: 0.7607 - val_loss: 0.2516 - val_accuracy: 0.9002 - val_f1_score: 0.7267
Epoch 6/40
297/297 [==============================] - 34s 115ms/ste

In [ ]:
# jedno initial_state = [c, h]

In [89]:
model.fit([x_pos_train, x_train], y_train, epochs=40, batch_size=512, validation_data=([x_pos_val, x_val], y_val))

Epoch 1/40
297/297 [==============================] - 27s 68ms/step - loss: 0.2903 - accuracy: 0.8953 - f1_score: 0.6489 - val_loss: 0.2876 - val_accuracy: 0.8931 - val_f1_score: 0.6915
Epoch 2/40
297/297 [==============================] - 21s 71ms/step - loss: 0.2384 - accuracy: 0.9133 - f1_score: 0.7451 - val_loss: 0.2762 - val_accuracy: 0.8912 - val_f1_score: 0.7298
Epoch 3/40
297/297 [==============================] - 22s 76ms/step - loss: 0.2210 - accuracy: 0.9167 - f1_score: 0.7614 - val_loss: 0.2500 - val_accuracy: 0.9023 - val_f1_score: 0.7430
Epoch 4/40
297/297 [==============================] - 23s 77ms/step - loss: 0.2148 - accuracy: 0.9178 - f1_score: 0.7634 - val_loss: 0.2491 - val_accuracy: 0.9040 - val_f1_score: 0.7540
Epoch 5/40
297/297 [==============================] - 25s 84ms/step - loss: 0.2123 - accuracy: 0.9184 - f1_score: 0.7617 - val_loss: 0.2446 - val_accuracy: 0.9023 - val_f1_score: 0.7281
Epoch 6/40
297/297 [==============================] - 26s 89ms/step - 

In [ ]:
x_train = df_train.drop(["label", "time", "energy_ma_4", "energy_ma_8", "energy_ma_12", "energy_ma_24", "energy_ma_36"], axis=1).to_numpy()
y_train = df_train["label"].to_numpy()
x_val = df_val.drop(["label", "time", "energy_ma_4", "energy_ma_8", "energy_ma_12", "energy_ma_24", "energy_ma_36"], axis=1).to_numpy()
y_val = df_val["label"].to_numpy()

In [ ]:
model = xgb.XGBClassifier(n_estimators=1000, max_depth=10, learning_rate=0.01, n_jobs=-1)

In [ ]:
model.fit(x_train, y_train, eval_set=[(x_val, y_val)], eval_metric='logloss', early_stopping_rounds=10)

In [ ]:
y_pred = model.predict(x_val)
accuracy_score(y_val, y_pred), f1_score(y_val, y_pred)